# Creating DataFrames
We will create `DataFrame` objects from other data structures in Python, by reading in a CSV file, and by querying a database.

## About the Data
In this notebook, we will be working with Earthquake data from September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Imports

In [1]:
import datetime
import numpy as np
import pandas as pd

## Creating a `Series`

In [2]:
np.random.seed(0) # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64

## Creating a `DataFrame` from a `Series`
Use the `to_frame()` method:

In [ ]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

## Creating a `DataFrame` from Python Data Structures
### From a dictionary of list-like structures
The dictionary values can be lists, numpy arrays, etc. as long as they have length (generators don't have length so we can't use them here):

In [ ]:
np.random.seed(0) # set seed so result is reproducible
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=datetime.date(2019, 4, 21),
        freq='1D',
        periods=5, 
        name='date'
    )
)

### From a list of dictionaries

In [ ]:
pd.DataFrame([
    {'mag' : 5.2, 'place' : 'California'},
    {'mag' : 1.2, 'place' : 'Alaska'},
    {'mag' : 0.2, 'place' : 'California'},
])

### From a list of tuples
This is equivalent to using `pd.DataFrame.from_records()`:

In [ ]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

In [ ]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_squared', 'n_cubed']
)

### From a NumPy array

In [ ]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)

## Creating a `DataFrame` by Reading in a CSV File

### Finding information on the file before reading it in
Before attempting to read in a file, we can use the command line to see important information about the file that may determine how we read it in. We can run command line code from Jupyter Notebooks (thanks to IPython) by using `!` before the code.

#### Number of lines (row count)
For example, we can find out how many lines are in the file by using the `wc` utility (word count) and counting lines in the file (`-l`). The file has 9,333 lines:

In [ ]:
!wc -l data/earthquakes.csv

#### File size
We can find the file size by using `ls` to list the files in the `data` directory, and passing in the flags `-lh` to include the file size in human readable format. Then we use `grep` to find the file in question. Note that the `|` passes the result of `ls` to `grep`. `grep` is used for finding items that match patterns.

This tells us the file is 3.4 MB:

In [ ]:
!ls -lh data | grep earthquakes.csv

We can even capture the result of a command and use it in our Python code:

In [ ]:
files = !ls -lh data
[file for file in files if 'earthquake' in file]

#### Examining a few rows
We can use `head` to look at the top `n` rows of the file. With the `-n` flag, we can specify how many. This shows use that the first row of the file contains headers and that it is comma-separated (just because the file extension is `.csv` doesn't mean they have to be comma-separated values):

In [ ]:
!head -n 2 data/earthquakes.csv

Just like `head` gives rows from the top, `tail` gives rows from the bottom. This can help us check that there is no extraneous data on the bottom of the field, like perhaps some metadata about the fields that actually isn't part of the data set:

In [ ]:
!tail -n 2 data/earthquakes.csv

#### Column count
We can use `awk` to find the column count. This is a Linux utility for pattern scanning and processing. The `-F` flag allows us to specify the delimiter (comma, in this case). Then we specify what to do for each record in the file. We choose to print `NF` which is a predefined variable whose value is the number of fields in the current record. Here, we say `exit` so that we print the number of fields (columns, here) in the first row of the file, then we stop. 

This tells us we have 26 data columns:

In [ ]:
!awk -F',' '{print NF; exit}' data/earthquakes.csv

Since we know the 1st line of the file had headers, and the file is comma-separated, we can also count the columns by using `head` to get headers and parsing them in Python:

In [ ]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

### Reading in the file
Our file is small in size, has headers in the first row, and is comma-separated, so we don't need to provide any additional arguments to read in the file with `pd.read_csv()`, but be sure to check the [documentation](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) for possible arguments:

In [ ]:
df = pd.read_csv('data/earthquakes.csv')

## Writing a `DataFrame` to a CSV File
Note that the index of `df` is just row numbers, so we don't want to keep it.

In [ ]:
df.to_csv('output.csv', index=False)

## Writing a `DataFrame` to a Database
Note the `if_exists` parameter. By default it will give you an error if you try to write a table that already exists. Here, we don't care if it is overwritten. Lastly, if we are interested in appending new rows, we set that to `'append'`.

In [ ]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    pd.read_csv('data/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

## Creating a `DataFrame` by Querying a Database
Using a SQLite database. Otherwise you need to install [SQLAlchemy](https://www.sqlalchemy.org/).

In [ ]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    tsunamis = pd.read_sql('SELECT * FROM tsunamis', connection)

tsunamis.head()